In [1]:
import pickle
from pathlib import Path

import numpy as np

In [2]:
log_dir = r"G:\projects\gfos\logs\pipeline\xla_ensemble\multiruns\2023-10-22_15-23-15"
output_files = list(Path(log_dir).glob("*/wandb/*/files/*.plk"))


In [5]:
output_files = [
    # "../../output/nlp_random_ue1gfayg.csv",
    r"G:\projects\gfos\logs\pipeline\nlp_ensemble\multiruns\2023-10-27_10-58-40\0\wandb\run-20231027_110222-0ytzj3l1\files\nlp_random_0ytzj3l1_logits.plk",
    r"G:\projects\gfos\logs\pipeline\nlp_ensemble\multiruns\2023-10-28_10-34-53\0\wandb\run-20231028_103842-r0srrmo6\files\nlp_random_r0srrmo6_logits.plk",
    r"G:\projects\gfos\logs\pipeline\nlp_ensemble\multiruns\2023-10-28_10-34-53\1\wandb\run-20231029_093944-q2gyfl1h\files\nlp_random_q2gyfl1h_logits.plk",
]

In [6]:
logits = [pickle.load(open(f, "rb")) for f in output_files]

In [7]:
results = {}

for logit in logits:
    for key, value in logit.items():
        if key not in results:
            results[key] = []
        # results[key].append((value - value.min()) / (value.max() - value.min()))
        results[key].append(value)

In [8]:
blend_results = {}

for key, value in results.items():
    r = np.stack(value, axis=0)
    blend_results[key] = np.argsort(np.mean(value, axis=0))

In [9]:
import matplotlib.pyplot as plt


rs = results["05ae41e26dd3c4c06390371a0423233c"]
axes = plt.subplots(5, 1, figsize=(10, 10))[1].flatten()
for i, ax in enumerate(axes):
    ax.hist(rs[i])


KeyError: '05ae41e26dd3c4c06390371a0423233c'

In [10]:
SOURCE = "nlp"
SEARCH = "random"


with open(f"../../output/{SOURCE}_{SEARCH}_blend_3_folds.csv", "w") as f:
    f.write("ID,TopConfigs\n")
    for k, v in blend_results.items():
        model_id = f"layout:{SOURCE}:{SEARCH}:"+k
        values = ";".join([str(i) for i in v])
        f.write(f"{model_id},{values}\n")
